In [22]:
import sys
sys.path.append('C:\\Users\\hp\\AppData\\Local\\Programs\\Python\\Python37\\Lib\\site-packages')
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline
from plotly import graph_objs as go
import plotly.figure_factory as ff
from tqdm import tqdm
import tensorflow as tf

#import libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, roc_auc_score,accuracy_score
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD,Adam
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
import keras
# Conv1D + LSTM
from keras.layers.convolutional import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D
from keras.layers import GlobalMaxPooling1D, Flatten, Bidirectional, SpatialDropout1D, LSTM, Dense, Flatten

from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import ResNet50

from keras.layers.embeddings import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline

from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from keras.models import load_model



In [23]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
train_path=input("Enter the Dataset Path for train:  \n")
test_path=input("Enter the Dataset Path for test:  \n")
valid_path=input("Enter the Dataset Path for valid:  \n")

Enter the Dataset Path for train:  
jigsaw-toxic-comment-train.csv
Enter the Dataset Path for test:  
test.csv
Enter the Dataset Path for valid:  
validation.csv


In [5]:
inp=int(input("Enter the Type of dataset:\n1.Text\n2.CSV\n3.Excel\n4.Image\n5.Audio\n"))

Enter the Type of dataset:
1.Text
2.CSV
3.Excel
4.Image
5.Audio
2


In [6]:
if inp==1:
    delim=input("Enter the delimiter present for given dataset:")
    df_train = pd.read_csv(train_path,delimiter=delim)
    df_test = pd.read_csv(test_path,delimiter=delim)
    df_valid = pd.read_csv(valid_path,delimiter=delim)
elif inp==2:
    df_train = pd.read_csv(train_path)
    df_test = pd.read_csv(test_path)
    df_valid = pd.read_csv(valid_path)
elif inp==3:  
    df_train = pd.read_excel(train_path)
    df_test = pd.read_excel(test_path)
    df_valid =pd.read_excel(valid_path)
elif inp==4:
    Image_Width = 512
    Image_Height = 512
    Image_Size = (Image_Width, Image_Height)
    train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,)
    test_generator = test_datagen.flow_from_directory(path,
                                                  target_size=Image_Size,
                                                  batch_size = 32,
                                                  class_mode='categorical')
    test_datagen = ImageDataGenerator(rescale=1./255)
elif inp==5:
    print(5)

In [7]:
df_train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [8]:
df_train.head(5)

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [9]:
df_train = df_train.loc[:12000,:]
df_train.shape

(12001, 3)

In [10]:
df_train['comment_text'].apply(lambda x:len(str(x).split())).max()

1403

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_train.comment_text.values, df_train.toxic.values, 
                                                  stratify=df_train.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [12]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((9600,), (2401,), (9600,), (2401,))

In [13]:
DLmodels = []
DLhistory=[]
AUCVal=[]

# Deep Learning Model 1 -ANN

In [13]:
learning_rate=0.001
batch_size=512
epochs = 1000

model_save = ModelCheckpoint('./JigsawToxic.h5', 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'val_loss', 
                             mode = 'min', verbose = 1)
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 50, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.75, 
                              patience = 10, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [14]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 1500

token.fit_on_texts(list(X_train) + list(X_test))
xtrain_seq = token.texts_to_sequences(X_train)
xvalid_seq = token.texts_to_sequences(X_test)

#zero pad the sequences
X_train_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [15]:
def Create_Model_ANN(num_columns, num_labels,learning_rate):
    model = Sequential()
   
    model.add(Dense(64,input_dim=num_columns, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_labels,activation='sigmoid'))
    
    adam = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #adam = Adam(lr=learning_rate)
    
    model.compile(loss='binary_crossentropy', optimizer=adam,metrics=[keras.metrics.AUC(name='auc')])
    
    return model

model = Create_Model_ANN(X_train_pad.shape[1], 1,learning_rate)

C:\Users\hp\AppData\Local\Programs\Python\Python37\Lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [16]:
input_size=X_train_pad.shape[1]

model1 = keras.Sequential()

w0=model.layers[0].get_weights()[0][:,:]
b0=model.layers[0].get_weights()[1]

model1.add(Dense(64,input_dim=input_size, activation='relu'))
model1.layers[0].set_weights([w0, b0])

# Add trained layers (with weight)
for layer in model.layers[1:7]:
    model1.add(layer)
    
# add output layer  sigmoid or softmax..    
model1.add(Dense(1,activation='sigmoid'))
#model_new.layers[0].set_weights([w0, b0])

adam = Adam(lr=learning_rate)
model1.compile(loss='binary_crossentropy', optimizer=adam,metrics=[keras.metrics.AUC(name='auc')])

In [17]:
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                96064     
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)               

In [18]:
history1=model1.fit(X_train_pad, y_train,validation_data=(xvalid_pad,y_test),callbacks = [model_save, early_stop, reduce_lr],
          verbose=1,
          batch_size=batch_size,
          epochs=epochs)


Epoch 1/1000
18/19 [===========================>..] - ETA: 0s - loss: 21.1109 - auc: 0.5021
Epoch 1: val_loss improved from inf to 2.15358, saving model to .\JigsawToxic.h5
19/19 [==============================] - 11s 188ms/step - loss: 20.3846 - auc: 0.5029 - val_loss: 2.1536 - val_auc: 0.5584 - lr: 0.0010
Epoch 2/1000
18/19 [===========================>..] - ETA: 0s - loss: 1.6975 - auc: 0.5340
Epoch 2: val_loss improved from 2.15358 to 1.09560, saving model to .\JigsawToxic.h5
19/19 [==============================] - 1s 63ms/step - loss: 1.6665 - auc: 0.5327 - val_loss: 1.0956 - val_auc: 0.5369 - lr: 0.0010
Epoch 3/1000
18/19 [===========================>..] - ETA: 0s - loss: 0.6076 - auc: 0.5954
Epoch 3: val_loss improved from 1.09560 to 0.70540, saving model to .\JigsawToxic.h5
19/19 [==============================] - 1s 63ms/step - loss: 0.6036 - auc: 0.5960 - val_loss: 0.7054 - val_auc: 0.5594 - lr: 0.0010
Epoch 4/1000
19/19 [==============================] - ETA: 0s - loss: 0.4

Epoch 30/1000
18/19 [===========================>..] - ETA: 0s - loss: 0.1062 - auc: 0.9752
Epoch 30: val_loss did not improve from 0.63392
19/19 [==============================] - 1s 54ms/step - loss: 0.1070 - auc: 0.9748 - val_loss: 0.9588 - val_auc: 0.5641 - lr: 5.6250e-04
Epoch 31/1000
19/19 [==============================] - ETA: 0s - loss: 0.1043 - auc: 0.9771
Epoch 31: val_loss did not improve from 0.63392
19/19 [==============================] - 1s 67ms/step - loss: 0.1043 - auc: 0.9771 - val_loss: 0.9625 - val_auc: 0.5586 - lr: 5.6250e-04
Epoch 32/1000
19/19 [==============================] - ETA: 0s - loss: 0.1055 - auc: 0.9755
Epoch 32: val_loss did not improve from 0.63392
19/19 [==============================] - 1s 70ms/step - loss: 0.1055 - auc: 0.9755 - val_loss: 1.0277 - val_auc: 0.5487 - lr: 5.6250e-04
Epoch 33/1000
18/19 [===========================>..] - ETA: 0s - loss: 0.0995 - auc: 0.9785
Epoch 33: val_loss did not improve from 0.63392
19/19 [======================

## Evaluation

In [19]:
y_pred = model1.predict(xvalid_pad)
AUC1 = metrics.roc_auc_score(y_test,y_pred)
print("AUC: {:.3f}".format(AUC1))

AUC: 0.572


In [20]:
scores_model = []
scores_model.append({'Model': 'ANN','AUC_Score': AUC1})
model1.save('ANN2Model.h5')
hist_df1 = pd.DataFrame(history1.history)
hist_df1.to_csv('History1.csv')

INFO:tensorflow:Assets written to: ram://97bcb670-9830-487a-a45b-664fcdbc6cc9/assets


In [21]:
DLmodels.append(model1)
DLhistory.append(history1)
AUCVal.append(AUC1)

# Deep Learning Model 2 -SimpleRNN

In [22]:
%%time
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    model2 = Sequential()
    model2.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model2.add(SimpleRNN(100))
    
    model2.add(Dense(1, activation='sigmoid'))
    model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 simple_rnn (SimpleRNN)      (None, 100)               40100     
                                                                 
 dense_10 (Dense)            (None, 1)                 101       
                                                                 
Total params: 13,089,301
Trainable params: 13,089,301
Non-trainable params: 0
_________________________________________________________________
Wall time: 1.1 s


In [23]:
history2=model2.fit(X_train_pad,
          y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's


Epoch 1/5
150/150 [==============================] - 315s 2s/step - loss: 0.3074 - accuracy: 0.8996
Epoch 2/5
150/150 [==============================] - 204s 1s/step - loss: 0.1169 - accuracy: 0.9604
Epoch 3/5
150/150 [==============================] - 211s 1s/step - loss: 0.0148 - accuracy: 0.9956
Epoch 4/5
150/150 [==============================] - 213s 1s/step - loss: 0.0037 - accuracy: 0.9993
Epoch 5/5
150/150 [==============================] - 214s 1s/step - loss: 0.1636 - accuracy: 0.9377


## Evaluation

In [26]:
y_pred2 = model2.predict(xvalid_pad)
AUC2 = metrics.roc_auc_score(y_test,y_pred2)
print("AUC: {:.3f}".format(AUC2))


AUC: 0.767


In [27]:
scores_model = []
scores_model.append({'Model': 'SimpleRNN','AUC_Score': AUC2})
model2.save('SimpleRNN2Model.h5')
hist_df2 = pd.DataFrame(history2.history)
hist_df2.to_csv('History2.csv')

INFO:tensorflow:Assets written to: ram://b79b1f38-2ba9-46a8-8214-e4c4b0a150a7/assets


In [28]:
DLmodels.append(model2)
DLhistory.append(history2)
AUCVal.append(AUC2)

## Deep Learning Model-3 LSTM

In [33]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [08:38, 4239.09it/s]


Found 2196016 word vectors.


In [34]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████████████████████████████████████████████| 43496/43496 [00:19<00:00, 2283.75it/s]


In [38]:
%%time
with strategy.scope():
    
    # A simple LSTM with glove embeddings and one dense layer
    model3 = Sequential()
    model3.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model3.add(LSTM(100,dropout=0.3, recurrent_dropout=0.3))
    model3.add(Dense(1, activation='sigmoid'))
    model3.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
model3.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1500, 300)         13049100  
                                                                 
 lstm_2 (LSTM)               (None, 100)               160400    
                                                                 
 dense_13 (Dense)            (None, 1)                 101       
                                                                 
Total params: 13,209,601
Trainable params: 160,501
Non-trainable params: 13,049,100
_________________________________________________________________
Wall time: 2.39 s


In [40]:
history3 = model3.fit(X_train_pad,y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 20138s 134s/step - loss: 0.2059 - accuracy: 0.9319
Epoch 2/5
150/150 [==============================] - 7317s 48s/step - loss: 0.1395 - accuracy: 0.9513
Epoch 3/5
150/150 [==============================] - 7040s 47s/step - loss: 0.1216 - accuracy: 0.9565
Epoch 4/5
150/150 [==============================] - 7102s 47s/step - loss: 0.1138 - accuracy: 0.9599
Epoch 5/5
150/150 [==============================] - 20984s 141s/step - loss: 0.1022 - accuracy: 0.9623


## Evaluation

In [41]:
y_pred3 = model3.predict(xvalid_pad)
AUC3 = metrics.roc_auc_score(y_test,y_pred3)
print("AUC: {:.3f}".format(AUC3))

AUC: 0.978


In [42]:
scores_model = []
scores_model.append({'Model': 'LSTM','AUC_Score': AUC3})
model3.save('LSTM2Model.h5')
hist_df3 = pd.DataFrame(history3.history)
hist_df3.to_csv('History3.csv')

INFO:tensorflow:Assets written to: ram://9d5d7c44-b93d-4853-a95e-67fcad68f81f/assets


In [43]:
DLmodels.append(model3)
DLhistory.append(history3)
AUCVal.append(AUC3)

## Deep Learning Model 4:GRU

In [35]:
%%time
with strategy.scope():
    # GRU with glove embeddings and two dense layers
    model4 = Sequential()
    model4.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model4.add(SpatialDropout1D(0.3))
    model4.add(GRU(300))
    model4.add(Dense(1, activation='sigmoid'))
    model4.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])   
    
model4.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1500, 300)         13049100  
                                                                 
 spatial_dropout1d (SpatialD  (None, 1500, 300)        0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 300)               540900    
                                                                 
 dense (Dense)               (None, 1)                 301       
                                                                 
Total params: 13,590,301
Trainable params: 541,201
Non-trainable params: 13,049,100
_________________________________________________________________
Wall time: 6.73 s


In [ ]:
history4=model4.fit(X_train_pad, y_train, epochs=5, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5
150/150 [==============================] - 34011s 228s/step - loss: 0.1980 - accuracy: 0.9334
Epoch 2/5
150/150 [==============================] - 31831s 212s/step - loss: 0.1281 - accuracy: 0.9518
Epoch 3/5
118/150 [======================>.......] - ETA: 2:46:38 - loss: 0.1032 - accuracy: 0.9620

## Evaluation

In [ ]:
y_pred4 = model4.predict(xvalid_pad)
AUC4 = metrics.roc_auc_score(y_test,y_pred4)
print("AUC: {:.3f}".format(AUC4))

In [ ]:
scores_model = []
scores_model.append({'Model': 'GRU','AUC_Score': AUC4})
model3.save('GRU2Model.h5')
hist_df3 = pd.DataFrame(history4.history)
hist_df3.to_csv('History4.csv')

In [ ]:
DLmodels.append(model4)
DLhistory.append(history4)
AUCVal.append(AUC4)

## Deep Learning Model 5: BiDirectional RNN

In [ ]:
%%time
with strategy.scope():
    # A simple bidirectional LSTM with glove embeddings and one dense layer
    model5 = Sequential()
    model5.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
    model5.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

    model5.add(Dense(1,activation='sigmoid'))
    model5.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    
    
model5.summary()

In [ ]:
history5=model5.fit(X_train_pad, y_train, nb_epoch=5, batch_size=64*strategy.num_replicas_in_sync)

In [ ]:
y_pred5 = model5.predict(xvalid_pad)
AUC5 = metrics.roc_auc_score(y_test,y_pred5)
print("AUC: {:.3f}".format(AUC5))

In [ ]:
scores_model = []
scores_model.append({'Model': 'BiderctionalRNN','AUC_Score': AUC5})
model1.save('BiRNN2Model.h5')
hist_df5 = pd.DataFrame(history5.history)
hist_df5.to_csv('History5.csv')

In [ ]:
DLmodels.append(model5)
DLhistory.append(history5)
AUCVal.append(AUC5)

## Models Comparison

In [ ]:
# Visualization of Results obtained from various Deep learning models
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.style.background_gradient(cmap='Blues')

## Best Model Visualization

### Training Curves

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history4.epoch[-1]+1),history4.history['val_auc'],label='val_auc')
plt.plot(range(history4.epoch[-1]+1),history4.history['auc'],label='auc')
plt.title('auc'); plt.xlabel('Epoch'); plt.ylabel('auc');plt.legend(); 
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(history4.epoch[-1]+1),history4.history['val_loss'],label='Val_loss')
plt.plot(range(history4.epoch[-1]+1),history4.history['loss'],label='loss')
plt.title('loss'); plt.xlabel('Epoch'); plt.ylabel('loss');plt.legend(); 
plt.show()

### ROC Curve

In [ ]:
def generate_results(y_test, y_score):
    # print(y_score)
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
generate_results(y_test, y_pred)